In [4]:
#Machine learning, array and dataframe libraries.

from numpy.random import seed
seed(1)

import numpy as np
import pandas as pd

import tensorflow
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Cleaned Data import 

In [5]:
import math

#To classify donation in ranges.
def roundNum(x):
    
    max = int(math.ceil((x+0.1)/1000)) *1000
    low = int(math.floor(x/1000)) *1000
    return f'[{low} - {max})'

In [2]:
#Import data from S3
cleanedElection_df = pd.read_csv('https://election-data-2020-red-raiders.s3.us-east-2.amazonaws.com/combinedClean.csv')

#Filter dataframe for the coluns we want.
cleanedElection_df = cleanedElection_df[['Zip','Occupation','Amount','Campaign']]

#declare data types
cleanedElection_df['Zip'] = cleanedElection_df['Zip'].astype(str)
cleanedElection_df['Occupation'] = cleanedElection_df['Occupation'].astype(str)
cleanedElection_df['Campaign'] = cleanedElection_df['Campaign'].astype(str)
cleanedElection_df['Amount'] = cleanedElection_df['Amount'].astype(int)

cleanedElection_df.head()

NameError: name 'pd' is not defined

In [3]:
#Loop to clasify values in ranges.
for index, row in cleanedElection_df.iterrows():
    
    cleanedElection_df.loc[index, 'Amount'] = roundNum(row['Amount'])

cleanedElection_df.head()

NameError: name 'cleanedElection_df' is not defined

# Categorical data encoding, train,test split and numerical data scaling

# Categorical encoding

In [5]:
#Label-encode data set
label_encoder = LabelEncoder()

encoded_df = pd.DataFrame()

for column in cleanedElection_df[['Zip','Occupation','Amount','Campaign']]:
    
    print(column)
    label_encoder.fit(cleanedElection_df[column])

    encoded_df[column] = label_encoder.transform(cleanedElection_df[column])


Zip


In [ ]:
#Original dataframe
cleanedElection_df.head()

In [ ]:
#Encoded data.
encoded_df.head()

In [8]:
#Separate outcome from data 
X = encoded_df.drop('Campaign', axis = 1)
y = encoded_df['Campaign']
print(X.shape, y.shape)

(9749263, 3) (9749263,)


## Train - Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

## Numerical Scaling

In [10]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# One hot encoding

In [11]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

#  Model Set up

In [12]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=150, activation='relu', input_dim=3)) #3 inputs 
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax')) #2 outcomes.

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
228499/228499 - 186s - loss: 0.4059 - accuracy: 0.8228
Epoch 2/10
228499/228499 - 188s - loss: 0.4032 - accuracy: 0.8229
Epoch 3/10
228499/228499 - 189s - loss: 0.4023 - accuracy: 0.8229
Epoch 4/10
228499/228499 - 188s - loss: 0.4016 - accuracy: 0.8230
Epoch 5/10
228499/228499 - 188s - loss: 0.4016 - accuracy: 0.8230
Epoch 6/10
228499/228499 - 188s - loss: 0.4017 - accuracy: 0.8230
Epoch 7/10
228499/228499 - 190s - loss: 0.4002 - accuracy: 0.8230
Epoch 8/10
228499/228499 - 188s - loss: 0.4016 - accuracy: 0.8230
Epoch 9/10
228499/228499 - 188s - loss: 0.4010 - accuracy: 0.8230
Epoch 10/10
228499/228499 - 188s - loss: 0.4010 - accuracy: 0.8230


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 150)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 26,002
Trainable params: 26,002
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

76167/76167 - 39s - loss: 0.4023 - accuracy: 0.8230
Normal Neural Network - Loss: 0.4023486077785492, Accuracy: 0.8230278491973877


# Predictions

In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [18]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['Democrat' 'Republican' 'Democrat' 'Democrat' 'Democrat' 'Democrat'
 'Republican' 'Democrat' 'Republican' 'Democrat']
Actual Labels: [0, 1, 0, 0, 0, 0, 1, 0, 1, 0]


In [19]:
cleanedElection_df.head()

,Zip,Occupation,Amount,Campaign
0,35205,Retired,[0 - 1000),Democrat
1,36695,Employed,[0 - 1000),Democrat
2,35213,Retired,[0 - 1000),Democrat
3,35242,Employed,[0 - 1000),Democrat
4,35242,Employed,[0 - 1000),Democrat


In [20]:
# Save the model
model.save("election_model.h5")